In [4]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm, gaussian_kde

# Generate synthetic data: two normal distributions with different means
# np.random.seed(42)
x_samples = np.random.normal(loc=0, scale=4, size=30*9)  # Sample from N(0, 1)
y_samples = np.random.normal(loc=1, scale=4, size=100*100)  # Sample from N(1, 1)

# Define CDF for integral-based CRPS
def cdf_normal(x, mean, std):
    return norm.cdf(x, loc=mean, scale=std)

# Integral-based CRPS computation (using Gaussian distributions as an example)
def crps_integral(mean_x, std_x, mean_y, std_y):
    def integrand(z):
        cdf_x = cdf_normal(z, mean_x, std_x)
        cdf_y = cdf_normal(z, mean_y, std_y)
        return (cdf_x - cdf_y) ** 2
    result, _ = quad(integrand, -np.inf, np.inf)
    return result

# Compute CRPS using integral-based method
mean_x, std_x = np.mean(x_samples), np.std(x_samples)
mean_y, std_y = np.mean(y_samples), np.std(y_samples)
crps_integral_value = crps_integral(mean_x, std_x, mean_y, std_y)
print(f"CRPS (Integral-based): {crps_integral_value}")

# Expectation-based CRPS computation using empirical samples
def crps_expectation_based(x_samples, y_samples):
    N = len(x_samples)
    M = len(y_samples)
    
    # First term: E[|X - Y|]
    term1 = np.mean([np.abs(x_i - y_j) for x_i in x_samples for y_j in y_samples])
    
    # Second term: E[|X - X'|]
    term2 = np.mean([np.abs(x_i - x_j) for x_i in x_samples for x_j in x_samples])
    
    # Third term: E[|Y - Y'|]
    term3 = np.mean([np.abs(y_i - y_j) for y_i in y_samples for y_j in y_samples])
    
    # CRPS empirical formula
    crps_empirical = term1 - 0.5 * term2 - 0.5 * term3
    return crps_empirical

# Compute CRPS using expectation-based method
crps_expectation_value = crps_expectation_based(x_samples, y_samples)
print(f"CRPS (Expectation-based): {crps_expectation_value}")

# Compare results
print(f"Difference between Integral and Expectation CRPS: {abs(crps_integral_value - crps_expectation_value)}")

CRPS (Integral-based): 0.12212035370510295
CRPS (Expectation-based): 0.11877613678503574
Difference between Integral and Expectation CRPS: 0.003344216920067214
